<a href="https://colab.research.google.com/github/StarNJey/localnotebook/blob/main/Local_Court_notebook_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <Chatbot 우주 by C.H.PARK>
인터넷에 연결되지 않은 컴퓨터에서도 NotebookLM과 같은 기능을 구현할 수 있을까요?

CPU만 사용할 수 있는 환경에서 실험을 해보니 결과가 괜찮아 예제를 만들어 보았습니다. 아래 코드를 순차적으로 실행한 후에 마지막으로 리턴된 값(두 개의 링크 중 첫 번재 링크)을 누르면 챗봇 창이 뜹니다.


* 아래 코드는 인터넷 + colab의 클라우드의 GPU를 활용합니다. 같은 결과를 GPU 없는 로컬 PC환경에서도 구현하였습니다(다만 CPU만 사용하는 특성상 아래 예제보다 속도가 훨씬 느립니다).

* 핵심 LLM은 LG에서 개발한 EXAONE 1.2B를 사용하였습니다. 1.2B도 성능이 꽤 괜찮아 32B를 사용하면 훨씬 더 좋은 결과가 나올 것 같습니다.

* 청크 max 크기를 크게 하고, 청크 하나에 들어가는 문장 수를 늘리면 어느 정도까지는 성능이 올라갑니다. 다만 너무 키우면 VRAM 부족으로 out of memory 에러가 발생합니다.

* 답변이 늦을 수 있지만, 궁금한 사항이 있으면 innerpeace.chp@gmail.com으로 연락주세요!

In [ ]:
!pip install streamlit pyngrok transformers sentence-transformers scikit-learn pdfplumber PyMuPDF PyPDF2 kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.5/35.5 MB 60.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 123.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━

In [ ]:
!git clone https://github.com/StarNJey/localnotebook.git

Cloning into 'localnotebook'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (9/9), 12.67 KiB | 4.22 MiB/s, done.


In [ ]:
%cd localnotebook/

/content/localnotebook/localnotebook


In [ ]:
from pyngrok import ngrok

In [ ]:
ngrok.set_auth_token("aRp7")

In [ ]:
import os, sys, subprocess, threading, torch
from pyngrok import ngrok

# 1) GPU 런타임 확인
assert torch.cuda.is_available(), "런타임 유형을 GPU로 변경하세요!"

# 2) GPU 디바이스 지정
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128,garbage_collection_threshold:0.6"

# 3) Python 인터프리터 경로
PY = sys.executable

def run_streamlit():
    # 4) Streamlit 실행 시 동일 Python 인터프리터와 env 전달
    subprocess.run(
        [PY, "-u", "-m", "streamlit", "run", "app.py", "--server.port", "8501"],
        env=os.environ
    )

# 5) Streamlit 서버 백그라운드 스레드로 실행
thread = threading.Thread(target=run_streamlit, daemon=True)
thread.start()

# 6) ngrok 터널 생성
public_url = ngrok.connect(8501, "http")
print("Streamlit URL:", public_url)


Streamlit URL: NgrokTunnel: "https://e1e07815e375.ngrok-free.app" -> "http://localhost:8501"
